In [1]:
# Importing 
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

pd.options.mode.chained_assignment = None # default='warn'
%matplotlib inline

In [2]:
# reading Melbourne housing data
house_data = pd.read_csv("melb_data.csv")

In [3]:
house_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [4]:
house_data.shape

(13580, 21)

In [5]:
house_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,13580.0,2.937997e+00,0.955748,1.00000,2.000000,3.000000,3.000000e+00,1.000000e+01
Price,13580.0,1.075684e+06,639310.724296,85000.00000,650000.000000,903000.000000,1.330000e+06,9.000000e+06
Distance,13580.0,1.013778e+01,5.868725,0.00000,6.100000,9.200000,1.300000e+01,4.810000e+01
Postcode,13580.0,3.105302e+03,90.676964,3000.00000,3044.000000,3084.000000,3.148000e+03,3.977000e+03
Bedroom2,13580.0,2.914728e+00,0.965921,0.00000,2.000000,3.000000,3.000000e+00,2.000000e+01
Bathroom,13580.0,1.534242e+00,0.691712,0.00000,1.000000,1.000000,2.000000e+00,8.000000e+00
Car,13518.0,1.610075e+00,0.962634,0.00000,1.000000,2.000000,2.000000e+00,1.000000e+01
Landsize,13580.0,5.584161e+02,3990.669241,0.00000,177.000000,440.000000,6.510000e+02,4.330140e+05
BuildingArea,7130.0,1.519676e+02,541.014538,0.00000,93.000000,126.000000,1.740000e+02,4.451500e+04
YearBuilt,8205.0,1.964684e+03,37.273762,1196.00000,1940.000000,1970.000000,1.999000e+03,2.018000e+03


In [6]:
house_data.columns.tolist()

['Suburb',
 'Address',
 'Rooms',
 'Type',
 'Price',
 'Method',
 'SellerG',
 'Date',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'CouncilArea',
 'Lattitude',
 'Longtitude',
 'Regionname',
 'Propertycount']

In [7]:
house_data.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

In [8]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [9]:
house_data.isnull().sum()

Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64

In [10]:
# Splitting the data and selecting relevant columns
feature_data = house_data[["Suburb", "Rooms", "Type", "SellerG", "Distance", "Postcode", "YearBuilt",
                           "Bedroom2", "Bathroom", "Car", "Landsize", "BuildingArea"]]
target_data = house_data["Price"]

In [11]:
feature_data.head()

,Suburb,Rooms,Type,SellerG,Distance,Postcode,YearBuilt,Bedroom2,Bathroom,Car,Landsize,BuildingArea
0,Abbotsford,2,h,Biggin,2.5,3067.0,NaN,2.0,1.0,1.0,202.0,NaN
1,Abbotsford,2,h,Biggin,2.5,3067.0,1900.0,2.0,1.0,0.0,156.0,79.0
2,Abbotsford,3,h,Biggin,2.5,3067.0,1900.0,3.0,2.0,0.0,134.0,150.0
3,Abbotsford,3,h,Biggin,2.5,3067.0,NaN,3.0,2.0,1.0,94.0,NaN
4,Abbotsford,4,h,Nelson,2.5,3067.0,2014.0,3.0,1.0,2.0,120.0,142.0


In [12]:
feature_data.isnull().sum()

Suburb             0
Rooms              0
Type               0
SellerG            0
Distance           0
Postcode           0
YearBuilt       5375
Bedroom2           0
Bathroom           0
Car               62
Landsize           0
BuildingArea    6450
dtype: int64

In [13]:
# Selecting only missing data
column_miss = feature_data.isnull().sum()
column_miss = column_miss[column_miss!=0]
column_miss

YearBuilt       5375
Car               62
BuildingArea    6450
dtype: int64

In [14]:
feature_data["Type"].value_counts()

h    9449
u    3017
t    1114
Name: Type, dtype: int64

In [15]:
feature_data.loc[feature_data["Type"]=="h"].isnull().sum()

Suburb             0
Rooms              0
Type               0
SellerG            0
Distance           0
Postcode           0
YearBuilt       4041
Bedroom2           0
Bathroom           0
Car               60
Landsize           0
BuildingArea    4589
dtype: int64

In [16]:
feature_data.loc[feature_data["Type"]=="u"].isnull().sum()

Suburb             0
Rooms              0
Type               0
SellerG            0
Distance           0
Postcode           0
YearBuilt        982
Bedroom2           0
Bathroom           0
Car                2
Landsize           0
BuildingArea    1433
dtype: int64

In [17]:
feature_data.loc[feature_data["Type"]=="t"].isnull().sum()

Suburb            0
Rooms             0
Type              0
SellerG           0
Distance          0
Postcode          0
YearBuilt       352
Bedroom2          0
Bathroom          0
Car               0
Landsize          0
BuildingArea    428
dtype: int64

In [18]:
feature_data.loc[feature_data["Type"]=="t"]["BuildingArea"]

21         NaN
38       225.0
42       134.0
48        90.0
56       133.0
         ...  
13437    215.0
13485      NaN
13504    138.0
13521      0.0
13526    128.0
Name: BuildingArea, Length: 1114, dtype: float64

In [19]:
# Classified data per Type for handling missing value
data_h = feature_data.loc[feature_data["Type"]=="h"]
data_u = feature_data.loc[feature_data["Type"]=="u"]
data_t = feature_data.loc[feature_data["Type"]=="t"]

In [20]:
# Imputing missing data
imputer = SimpleImputer()
feature_data.loc[feature_data.Type == "h", "Car"] = imputer.fit_transform(data_h["Car"].values.reshape(-1,1))
feature_data.loc[feature_data.Type == "h", "YearBuilt"] = imputer.fit_transform(data_h["YearBuilt"].values.reshape(-1,1))
feature_data.loc[feature_data.Type == "h", "BuildingArea"] = imputer.fit_transform(data_h["BuildingArea"].values.reshape(-1,1))
feature_data.loc[feature_data.Type == "u", "Car"] = imputer.fit_transform(data_u["Car"].values.reshape(-1,1))
feature_data.loc[feature_data.Type == "u", "YearBuilt"] = imputer.fit_transform(data_u["YearBuilt"].values.reshape(-1,1))
feature_data.loc[feature_data.Type == "u", "BuildingArea"] = imputer.fit_transform(data_u["BuildingArea"].values.reshape(-1,1))
feature_data.loc[feature_data.Type == "t", "YearBuilt"] = imputer.fit_transform(data_t["YearBuilt"].values.reshape(-1,1))
feature_data.loc[feature_data.Type == "t", "BuildingArea"] = imputer.fit_transform(data_t["BuildingArea"].values.reshape(-1,1))

In [21]:
feature_data.isnull().sum()

Suburb          0
Rooms           0
Type            0
SellerG         0
Distance        0
Postcode        0
YearBuilt       0
Bedroom2        0
Bathroom        0
Car             0
Landsize        0
BuildingArea    0
dtype: int64

In [22]:
feature_data.head()

,Suburb,Rooms,Type,SellerG,Distance,Postcode,YearBuilt,Bedroom2,Bathroom,Car,Landsize,BuildingArea
0,Abbotsford,2,h,Biggin,2.5,3067.0,1954.081176,2.0,1.0,1.0,202.0,176.866248
1,Abbotsford,2,h,Biggin,2.5,3067.0,1900.000000,2.0,1.0,0.0,156.0,79.000000
2,Abbotsford,3,h,Biggin,2.5,3067.0,1900.000000,3.0,2.0,0.0,134.0,150.000000
3,Abbotsford,3,h,Biggin,2.5,3067.0,1954.081176,3.0,2.0,1.0,94.0,176.866248
4,Abbotsford,4,h,Nelson,2.5,3067.0,2014.000000,3.0,1.0,2.0,120.0,142.000000


In [23]:
# Rounding "YearBuilt" and "BuildingArea" to one digit
feature_data["YearBuilt"] = round(feature_data["YearBuilt"],0)

In [24]:
feature_data["BuildingArea"] = round(feature_data["BuildingArea"],0)

In [25]:
feature_data.head()

,Suburb,Rooms,Type,SellerG,Distance,Postcode,YearBuilt,Bedroom2,Bathroom,Car,Landsize,BuildingArea
0,Abbotsford,2,h,Biggin,2.5,3067.0,1954.0,2.0,1.0,1.0,202.0,177.0
1,Abbotsford,2,h,Biggin,2.5,3067.0,1900.0,2.0,1.0,0.0,156.0,79.0
2,Abbotsford,3,h,Biggin,2.5,3067.0,1900.0,3.0,2.0,0.0,134.0,150.0
3,Abbotsford,3,h,Biggin,2.5,3067.0,1954.0,3.0,2.0,1.0,94.0,177.0
4,Abbotsford,4,h,Nelson,2.5,3067.0,2014.0,3.0,1.0,2.0,120.0,142.0


In [26]:
# Converting categorical data value to numerical data value
encoder = OrdinalEncoder()
feature_data["Suburb"] = encoder.fit_transform(feature_data[["Suburb"]])
feature_data["SellerG"] = encoder.fit_transform(feature_data[["SellerG"]])
feature_data["Type"] = encoder.fit_transform(feature_data[["Type"]])

In [27]:
feature_data.head()

,Suburb,Rooms,Type,SellerG,Distance,Postcode,YearBuilt,Bedroom2,Bathroom,Car,Landsize,BuildingArea
0,0.0,2,0.0,23.0,2.5,3067.0,1954.0,2.0,1.0,1.0,202.0,177.0
1,0.0,2,0.0,23.0,2.5,3067.0,1900.0,2.0,1.0,0.0,156.0,79.0
2,0.0,3,0.0,23.0,2.5,3067.0,1900.0,3.0,2.0,0.0,134.0,150.0
3,0.0,3,0.0,23.0,2.5,3067.0,1954.0,3.0,2.0,1.0,94.0,177.0
4,0.0,4,0.0,155.0,2.5,3067.0,2014.0,3.0,1.0,2.0,120.0,142.0


In [29]:
scaler = StandardScaler()
scaled = scaler.fit_transform(feature_data)
scaled

array([[-1.71982483, -0.98146337, -0.63200323, ..., -0.63584819,
        -0.08931566,  0.06219426],
       [-1.71982483, -0.98146337, -0.63200323, ..., -1.67699936,
        -0.10084297, -0.18720721],
       [-1.71982483,  0.06487613, -0.63200323, ..., -1.67699936,
        -0.10635603, -0.00651839],
       ...,
       [ 1.65763629,  0.06487613, -0.63200323, ...,  2.48760531,
        -0.03067672,  0.06219426],
       [ 1.65763629,  1.11121563, -0.63200323, ...,  3.52875648,
         0.0770786 ,  0.011296  ],
       [ 1.74622544,  1.11121563, -0.63200323, ..., -0.63584819,
        -0.04922066, -0.10322508]])

In [31]:
cols = feature_data.columns

In [32]:
feature_data = pd.DataFrame(scaled, columns=cols)

In [34]:
feature_data.head()

,Suburb,Rooms,Type,SellerG,Distance,Postcode,YearBuilt,Bedroom2,Bathroom,Car,Landsize,BuildingArea
0,-1.719825,-0.981463,-0.632003,-1.386938,-1.301485,-0.422415,-0.313374,-0.947035,-0.772376,-0.635848,-0.089316,0.062194
1,-1.719825,-0.981463,-0.632003,-1.386938,-1.301485,-0.422415,-2.094296,-0.947035,-0.772376,-1.676999,-0.100843,-0.187207
2,-1.719825,0.064876,-0.632003,-1.386938,-1.301485,-0.422415,-2.094296,0.088284,0.673367,-1.676999,-0.106356,-0.006518
3,-1.719825,0.064876,-0.632003,-1.386938,-1.301485,-0.422415,-0.313374,0.088284,0.673367,-0.635848,-0.116380,0.062194
4,-1.719825,1.111216,-0.632003,0.311181,-1.301485,-0.422415,1.665429,0.088284,-0.772376,0.405303,-0.109864,-0.026878


In [37]:
print("size of feature and target data is: ", feature_data.shape, target_data.shape)

size of feature and target data is:  (13580, 12) (13580,)
